In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.test.is_gpu_available():
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

from functools import partial
import numpy as np
from sklearn.datasets import load_sample_image
import tensorflow_datasets as tfds

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

So it looks like we're going to use `Xception` model. Let's start.

## transfer learning

### preprocess data

In [3]:
test_split, valid_split, train_split = tfds.Split.TRAIN.subsplit([10, 15, 75])

test_set_raw = tfds.load("tf_flowers", split=test_split, as_supervised=True)
valid_set_raw = tfds.load("tf_flowers", split=valid_split, as_supervised=True)
train_set_raw = tfds.load("tf_flowers", split=train_split, as_supervised=True)

In [4]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [5]:
def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

### load info

In [6]:
_, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

In [7]:
n_classes = info.features["label"].num_classes

In [8]:
n_classes

5

In [9]:
dataset_size = info.splits["train"].num_examples

In [10]:
dataset_size

3670

### load model

In [ ]:
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [ ]:
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)

In [ ]:
for layer in base_model.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)

### resnet

Let's try to use resnet. That's not easy - we don't know the correct learning rate and we get really low `val_accuracy`.

#### lr finder

In [21]:
def train_resnet(lr=.2, epochs=2):
    
    tf.keras.backend.clear_session()
    tf.random.set_seed(42)
    
    base_model_resnet = tf.keras.applications.ResNet101V2(weights="imagenet",
                                                              include_top=False)
    avg = tf.keras.layers.GlobalAveragePooling2D()(base_model_resnet.output)
    output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
    model_resnet = tf.keras.models.Model(inputs=base_model_resnet.input, outputs=output)
    
    for layer in base_model_resnet.layers:
        layer.trainable = False
    
    optimizer = tf.keras.optimizers.SGD(lr=lr, momentum=0.9, decay=0.01)
#     optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model_resnet.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
    
    history_resnet = model_resnet.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=epochs)

In [24]:
rates = [.1, .05, .001]

In [25]:
for lr in rates:
    print(f'lr={lr}')
    train_resnet(lr=lr, epochs=3)

lr=0.1
Train for 86 steps, validate for 17 steps
Epoch 1/3
86/86 [==============================] - 24s 284ms/step - loss: 1.9534 - accuracy: 0.7613 - val_loss: 7.4733 - val_accuracy: 0.6581
Epoch 2/3
86/86 [==============================] - 17s 202ms/step - loss: 1.1915 - accuracy: 0.8605 - val_loss: 5.1221 - val_accuracy: 0.7059
Epoch 3/3
86/86 [==============================] - 18s 206ms/step - loss: 0.7635 - accuracy: 0.8859 - val_loss: 4.9482 - val_accuracy: 0.7096
lr=0.05
Train for 86 steps, validate for 17 steps
Epoch 1/3
86/86 [==============================] - 25s 293ms/step - loss: 0.9091 - accuracy: 0.7852 - val_loss: 3.9208 - val_accuracy: 0.6673
Epoch 2/3
86/86 [==============================] - 18s 208ms/step - loss: 0.4961 - accuracy: 0.8710 - val_loss: 3.2275 - val_accuracy: 0.6783
Epoch 3/3
86/86 [==============================] - 18s 215ms/step - loss: 0.4262 - accuracy: 0.8924 - val_loss: 2.9868 - val_accuracy: 0.6838
lr=0.001
Train for 86 steps, validate for 17 step

#### training

In [35]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

base_model_resnet = tf.keras.applications.ResNet101V2(weights="imagenet",
                                                              include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model_resnet.output)
output = tf.keras.layers.Dense(n_classes, activation="softmax")(avg)
model_resnet = tf.keras.models.Model(inputs=base_model_resnet.input, outputs=output)

In [36]:
for layer in base_model_resnet.layers:
    layer.trainable = False

In [37]:
optimizer = tf.keras.optimizers.SGD(lr=0.09, momentum=0.9, decay=0.01)
# optimizer = tf.keras.optimizers.Adam(learning_rate=.007)
model_resnet.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [38]:
history_resnet = model_resnet.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)

Train for 86 steps, validate for 17 steps
Epoch 1/5
86/86 [==============================] - 26s 306ms/step - loss: 1.4257 - accuracy: 0.7740 - val_loss: 6.1327 - val_accuracy: 0.6875
Epoch 2/5
86/86 [==============================] - 19s 222ms/step - loss: 0.9116 - accuracy: 0.8579 - val_loss: 4.9151 - val_accuracy: 0.6728
Epoch 3/5
86/86 [==============================] - 19s 218ms/step - loss: 0.6592 - accuracy: 0.8888 - val_loss: 3.8833 - val_accuracy: 0.7647
Epoch 4/5
86/86 [==============================] - 18s 210ms/step - loss: 0.4408 - accuracy: 0.9157 - val_loss: 3.7745 - val_accuracy: 0.7482
Epoch 5/5
86/86 [==============================] - 18s 209ms/step - loss: 0.4263 - accuracy: 0.9110 - val_loss: 3.3897 - val_accuracy: 0.7647


In [39]:
for layer in base_model_resnet.layers:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=0.005, momentum=0.9,
                                 nesterov=True, decay=0.0005)
# optimizer = tf.keras.optimizers.Adam(learning_rate=.007/10)
model_resnet.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model_resnet.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=10)

Train for 86 steps, validate for 17 steps
Epoch 1/10
86/86 [==============================] - 63s 730ms/step - loss: 1.0866 - accuracy: 0.6853 - val_loss: 6.7450 - val_accuracy: 0.3493
Epoch 2/10
86/86 [==============================] - 47s 542ms/step - loss: 0.6007 - accuracy: 0.7994 - val_loss: 1.0123 - val_accuracy: 0.7408
Epoch 3/10
86/86 [==============================] - 48s 554ms/step - loss: 0.3685 - accuracy: 0.8586 - val_loss: 0.6103 - val_accuracy: 0.8180
Epoch 4/10
86/86 [==============================] - 47s 546ms/step - loss: 0.2885 - accuracy: 0.8968 - val_loss: 0.4172 - val_accuracy: 0.8529
Epoch 5/10
86/86 [==============================] - 47s 547ms/step - loss: 0.2300 - accuracy: 0.9179 - val_loss: 0.3621 - val_accuracy: 0.8529
Epoch 6/10
86/86 [==============================] - 47s 547ms/step - loss: 0.1846 - accuracy: 0.9371 - val_loss: 0.3030 - val_accuracy: 0.8860
Epoch 7/10
86/86 [==============================] - 47s 547ms/step - loss: 0.1463 - accuracy: 0.9477